In [ ]:
from infernal import feature_extraction as fe
from infernal import datastructures as ds
import spacy

In [ ]:
nlp = spacy.load('pt_core_news_lg')

In [ ]:
#!python -m spacy download pt_core_news_lg
#!pip install zss rdflib

In [1]:
texto = '''Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ). Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.

"Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano. É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú. "A reforma tributária tem importância muito maior que autonomia do Banco Central", comentou.

O projeto de autonomia do BC foi aprovado na terça-feira, 3, pelo Senado e agora precisa do aval dos deputados para virar lei. O texto mantém o controle dos preços como objetivo central, mas inclui ainda duas novas metas acessórias, sem prejuízo à principal: suavizar as flutuações do nível de atividade econômica e fomentar o pleno emprego no País. O governo concordou com a redação da proposta, apesar de o BC ser historicamente contrário a ampliar o escopo da atuação.

Maia já reclamou outras vezes da falta de empenho e atuação do governo para se aprovar a medida. Na semana passada, acusou o presidente do Banco Central, Roberto Campos Neto, de ter vazado informações sobre conversa que os dois tiveram no dia da decisão do Comitê de Política Monetária (Copom), que manteve a taxa Selic em 2% ao ano.

Ao jornal O Estado de S. Paulo, Maia criticou a articulação do presidente do BC em alertar sobre os reflexos para a economia da dificuldade do Congresso em avançar com as votações da pauta de ajuste fiscal. Segundo o presidente da Câmara, Campos Neto tentou fazer uma articulação política, sem combinar, o que não seria papel dele, mas dos ministros da Economia, Paulo Guedes, e da articulação política, Luiz Eduardo Ramos.

Nesta sexta-feira, o presidente da Câmara lembrou que havia uma proposta de autonomia do BC semelhante na Câmara, mas que não foi votada, e disse que não comentou até agora sobre o tema porque não foi procurado pelo governo para falar sobre o assunto. "Se eu conseguisse conversar com alguém do governo, eu poderia te responder, mas ninguém me procura. Não vou conversar com a imprensa antes de conversar com o governo", disse.

Como o Broadcast (sistema de notícias em tempo real do Grupo Estado) mostrou na quinta-feira, deputados já se articulam na Câmara para modificar o projeto aprovado pelo Senado. O partido Novo, por exemplo, quer enxugar a proposta que recebeu aval dos senadores para reduzir os chamados acessórios que foram colocados para o Banco Central.

Sobre a reforma tributária, Maia deu sinais de que quer aprovar o projeto antes de deixar a presidência da Casa e acredita que com acordo pode fazer isso rapidamente.'''

texto

'Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ). Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.\n\n"Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano. É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú. "A reforma tributária tem importância muito maior que autonomia do Banco Central", comentou.\n\nO projeto de autonomia do BC foi aprovado na terça-feira, 3, pelo Senado e agora precisa do aval dos deputados para virar lei. O texto mantém o controle dos preços como objetivo central, mas inclui ainda duas novas metas acessórias, sem prejuízo à principal: suav

In [ ]:
textdoc = nlp(texto)

In [ ]:
from itertools import tee

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)    
    next(b, None)
    return zip(a, b)

In [ ]:
list(enumerate(textdoc.sents))

In [ ]:
sentencas = []

add_asspas = False
for s in textdoc.sents:
    if s.text == '"':
        add_asspas = True
    else:
        if add_asspas:
            sentencas.append('"'+s.text)
            add_asspas = False
        else:
            sentencas.append(s.text)
        

In [ ]:
pares = [(s1,s2) for s1,s2 in pairwise(sentencas)]

pares

In [ ]:
extrator = fe.FeatureExtractor(both=True)

extrator.get_feature_names()

In [ ]:
from infernal.datastructures import Entailment 
s1,s2 = pares[0]


par = ds.Pair(s1,s2, 0, Entailment.none)

par

In [ ]:
s_ex = None
for i,s in enumerate(textdoc.sents):
    s_ex = s
    if i == 1:
        break
    print(dir(s))
    print(type(s))

dir(s_ex)

In [2]:
from text_document import TextDocument

td = TextDocument(texto)

In [11]:
from single_sentence import SingleSentence
for i,s in enumerate(td.anotated_doc.sents):
    if i == 1:
        break
    
ss = SingleSentence(i, s.text, s)

ss = td.sentences[0]
print(td.sentences[0].anotated.ents)
print(td.sentences[1].anotated.ents)
print(td.sentences[3].anotated.ents)

ss.set_named_entities()

[Jair Bolsonaro, Banco Central, BC, Câmara, Casa, Rodrigo Maia, DEM-RJ]
[Roberto Campos Neto]
[Banco]


IndexError: list index out of range

### Problema de segmentação de sentenças nas asspas que iniciam uma frase

Esse problema provavelmente, não dá pra resolver apenas alterando as marcações de início das sentenças, por que no caso de asspas que iniciam uma frase, o parser de dependências marca elas como ROOT e acho que isso faz com que a marcação fiquei incorreta, mas não se consiga alterar os marcadores de borda das sentenças. 

In [ ]:
def set_custom_boundaries(doc):        
    for token in doc[:-1]:        
        if token.text == "\"" and \
           token.is_sent_start and \
           token.is_sent_end:            
            doc[token.i+1].is_sent_start = False
    
    return doc

nlp.add_pipe(set_custom_boundaries, before="parser")


In [ ]:
#nlp.remove_pipe('set_custom_boundaries')


In [ ]:
from spacy.pipeline import SentenceSegmenter

for n,_ in nlp.pipeline:
    if 'sentencizer_hook' == n: 
        nlp.remove_pipe('sentencizer_hook')

def split_on_breaks(doc):
    start = 0
    seen_break = False
    for word in doc:
        if seen_break:
            yield doc[start:word.i-1]
            start = word.i
            seen_break = False
        elif word.text == '"' and \
             word.is_sent_start and \
             word.is_sent_end:
            seen_break = True
            #yield doc[start:word.i-1]
            #start = word.i+1
    if start < len(doc):
        yield doc[start:len(doc)]

sbd = SentenceSegmenter(nlp.vocab, strategy=split_on_breaks)
nlp.add_pipe(sbd, first=False)

def get_sentences(text):
    doc = nlp(text)
    return (list(doc.sents)) # convert to string if required.



In [ ]:
text = '''"É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú. 
"A reforma tributária tem importância muito maior que autonomia do Banco Central", comentou.\nO projeto de autonomia do BC foi aprovado na terça-feira, 3, pelo Senado e agora precisa do aval dos deputados para virar lei.'''

get_sentences(text)
#doc = nlp(text)
#print("After:", [sent.text for sent in doc.sents])

In [ ]:
for token in doc:
    #if doc[token.i].is_sent_start and \
    #   doc[token.i].is_sent_end and \
    #   doc[token.i].text == '"': 
    if doc[token.i].text == '"': 
        print(doc[token.i-1].is_sent_start, doc[token.i-1].is_sent_end, doc[token.i-1].text)                        
        print(doc[token.i].is_sent_start,   doc[token.i].is_sent_end, doc[token.i].text)            
        print(doc[token.i+1].is_sent_start, doc[token.i+1].is_sent_end, doc[token.i+1].text)
        print(doc[token.i].dep_)
        print()


In [ ]:
for token in doc[:-1]:            
        print(doc[token.i].is_sent_start,   doc[token.i].is_sent_end, doc[token.i].text)                    
        print(doc[token.i].pos_)
        print(doc[token.i].dep_)
        print()
